### Use custom dataloader ###

In [15]:
from MoD44BLoader import MoD44BLoader
import numpy as np

In [16]:
loader = MoD44BLoader("h17v07/", "Percent_Tree_Cover")
data = loader.get_time_series()
print(data.shape)

(20, 4800, 4800)


#### Forecast on pixel time series data ####

In [18]:
from AdaptiveAutoRegForecaster import AdaptiveAutoRegForecaster
from prophet_forecaster import AdaptiveProphetForecaster
from movingavg import ExponentialMovingAverageForecaster

In [19]:
def get_ar_predictions(pixel_ts_data, training_size):
    forecaster = AdaptiveAutoRegForecaster()
    train_start = int(0)
    train_end = int(training_size * len(pixel_ts_data))
    actual = pixel_ts_data[train_end:]

    if np.all(pixel_ts_data == pixel_ts_data[0]):
        predicted = np.full((len(actual),), pixel_ts_data[0], dtype=np.float32)
    else:
        forecaster.fit(pixel_ts_data, (train_start, train_end))
        pred_diff = forecaster.forecast(steps=len(actual))
        predicted = forecaster.invert_difference(pred_diff, last_observed_value=pixel_ts_data[train_end-1])
    return predicted, actual

In [20]:
import pandas as pd

def get_prophet_predictions(pixel_ts_data, training_size, dataloader):
    dates = pd.DatetimeIndex(dataloader.get_time_datetimes())
    forecaster = AdaptiveProphetForecaster(
    seasonality_mode='additive',
    yearly_seasonality=False,  # Disable for annual data
        changepoint_prior_scale=0.05
    )
    train_start = int(0)
    train_end = int(training_size * len(pixel_ts_data))
    forecaster.fit(pixel_ts_data, dates=dates, train_range=(train_start, train_end))
    actual = pixel_ts_data[train_end:]
    predicted = forecaster.forecast(steps=len(actual), freq='YS')
    return predicted, actual

def rmse(predictions, targets):
    return ((predictions - targets) ** 2).mean() ** 0.5

In [22]:


def get_ema_predictions(pixel_ts_data, training_size, alpha=0.3):
    forecaster = ExponentialMovingAverageForecaster(alpha=alpha)

    train_end = int(training_size * len(pixel_ts_data))
    train_data = pixel_ts_data[:train_end]

    forecaster.fit(train_data)

    actual = pixel_ts_data[train_end:]
    predicted = forecaster.forecast(steps=len(actual))

    return predicted, actual


In [23]:
pixel_ts_data = loader.get_pixel_time_series(4225, 34)
training_size = 0.75
results_df = pd.DataFrame(columns=['Pixel', 'rmse_ar', 'rmse_prophet', 'rmse_ema'])
height = data.shape[0]
width = data.shape[1]
n_iterations = 100

In [24]:


# # Sample random pixels and compute RMSE for both models

# for i in range(n_iterations):
#     pixel_row = 4225+i
#     pixel_col = 34+i
#     pixel_ts_data = loader.get_pixel_time_series(pixel_row, pixel_col)

#     pred_ar, actual_ar = get_ar_predictions(pixel_ts_data, training_size)
#     rmse_ar = rmse(pred_ar, actual_ar)

#     pred_prophet, actual_prophet = get_prophet_predictions(pixel_ts_data, training_size, loader)
#     rmse_prophet = rmse(pred_prophet, actual_prophet)

#     results_df = pd.concat([results_df, pd.DataFrame({'Pixel': [f'({pixel_row}, {pixel_col})'], 'rmse_ar': [rmse_ar], 'rmse_prophet': [rmse_prophet]})], ignore_index=True)

for i in range(n_iterations):
    pixel_row = 4225 + i
    pixel_col = 34 + i
    pixel_ts_data = loader.get_pixel_time_series(pixel_row, pixel_col)

    # AR
    pred_ar, actual_ar = get_ar_predictions(pixel_ts_data, training_size)
    rmse_ar = rmse(pred_ar, actual_ar)

    # Prophet
    pred_prophet, actual_prophet = get_prophet_predictions(pixel_ts_data, training_size, loader)
    rmse_prophet = rmse(pred_prophet, actual_prophet)

    # Moving Average
    pred_ema, actual_ema = get_ema_predictions(pixel_ts_data, training_size, alpha=0.3)
    rmse_ema = rmse(pred_ema, actual_ema)


    results_df = pd.concat([
        results_df,
        pd.DataFrame({
            'Pixel': [f'({pixel_row}, {pixel_col})'],
            'rmse_ar': [rmse_ar],
            'rmse_prophet': [rmse_prophet],
            'rmse_ema': [rmse_ema],
        })
    ], ignore_index=True)


21:23:55 - cmdstanpy - INFO - Chain [1] start processing
21:23:55 - cmdstanpy - INFO - Chain [1] done processing
/var/folders/96/v548prc94dn_9tm9qd772j340000gn/T/ipykernel_67798/910794963.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([
21:23:55 - cmdstanpy - INFO - Chain [1] start processing
21:23:55 - cmdstanpy - INFO - Chain [1] done processing
21:23:55 - cmdstanpy - INFO - Chain [1] start processing
21:23:55 - cmdstanpy - INFO - Chain [1] done processing
21:23:55 - cmdstanpy - INFO - Chain [1] start processing
21:23:55 - cmdstanpy - INFO - Chain [1] done processing
21:23:55 - cmdstanpy - INFO - Chain [1] start processing
21:23:55 - cmdstanpy - INFO - Chain [1] done processing
21:23:55 - cmdstanpy - INFO - Ch

In [25]:
results_df

,Pixel,rmse_ar,rmse_prophet,rmse_ema
0,"(4225, 34)",8.201622,5.558440,5.610078
1,"(4226, 35)",5.295369,5.709910,6.207722
2,"(4227, 36)",11.096174,5.870573,6.025837
3,"(4228, 37)",5.463394,4.150785,3.998407
4,"(4229, 38)",18.183562,7.215479,8.236790
...,...,...,...,...
95,"(4320, 129)",11.941951,16.274131,10.393352
96,"(4321, 130)",11.060731,13.584364,9.427827
97,"(4322, 131)",16.505127,5.952379,5.035792
98,"(4323, 132)",17.537801,7.507727,6.658268


In [26]:
results_df['rmse_ar'].describe()

count    100.000000
mean      15.721806
std       43.883069
min        1.682375
25%        5.996921
50%        9.299712
75%       15.113998
max      442.007059
Name: rmse_ar, dtype: float64

In [27]:
results_df['rmse_prophet'].describe()

count    100.000000
mean       5.899722
std        3.351756
min        1.393260
25%        3.975651
50%        5.344740
75%        6.601187
max       17.612508
Name: rmse_prophet, dtype: float64

In [28]:
results_df['rmse_ema'].describe()

count    100.000000
mean       5.446652
std        1.907970
min        1.863270
25%        4.236454
50%        5.115721
75%        6.273972
max       11.185259
Name: rmse_ema, dtype: float64